# Design Patterns for Data Engineering Frameworks

## 🎯 Problem: Code Reusability and Extensibility

### The Framework Challenge

When building data engineering frameworks like Odibi, you need:
- **Extensibility**: Add new transforms, connectors, engines without modifying core code
- **Consistency**: Same interface for different implementations
- **Discoverability**: Find and register components dynamically
- **Testability**: Mock and test components in isolation

Design patterns solve these problems.

## 🦉 First Principles

1. **Design for Change**: Code will evolve; make change easy
2. **Favor Composition**: Combine small pieces over inheritance hierarchies
3. **Program to Interfaces**: Depend on contracts, not implementations
4. **Open/Closed Principle**: Open for extension, closed for modification

## Pattern 1: Registry Pattern

### ⚡ Example: Transform Registry

In [ ]:
from typing import Dict, Callable, Any
import pandas as pd

class TransformRegistry:
    """Central registry for data transforms."""
    
    _transforms: Dict[str, Callable] = {}
    
    @classmethod
    def register(cls, name: str):
        """Decorator to register a transform."""
        def decorator(func: Callable):
            cls._transforms[name] = func
            return func
        return decorator
    
    @classmethod
    def get(cls, name: str) -> Callable:
        """Get a registered transform."""
        if name not in cls._transforms:
            raise ValueError(f"Transform '{name}' not registered")
        return cls._transforms[name]
    
    @classmethod
    def list_transforms(cls) -> list[str]:
        """List all registered transforms."""
        return list(cls._transforms.keys())

# Register transforms using decorator
@TransformRegistry.register("uppercase")
def uppercase_transform(df: pd.DataFrame, column: str) -> pd.DataFrame:
    df = df.copy()
    df[column] = df[column].str.upper()
    return df

@TransformRegistry.register("add_prefix")
def add_prefix_transform(df: pd.DataFrame, column: str, prefix: str) -> pd.DataFrame:
    df = df.copy()
    df[column] = prefix + df[column]
    return df

# Use registered transforms
df = pd.DataFrame({"name": ["alice", "bob"]})

transform = TransformRegistry.get("uppercase")
result = transform(df, "name")
print(result)

print(f"\nAvailable transforms: {TransformRegistry.list_transforms()}")

### 🔍 Odibi Analysis: Registry Pattern

**Where used:**
- Transform registry for data transformations
- Function registry for custom operations
- Connector registry for different data sources

**Benefits:**
- Add new transforms without modifying core
- Discover available transforms at runtime
- Decouple registration from usage

## Pattern 2: Factory Pattern

### ⚡ Example: Connection Factory

In [ ]:
from abc import ABC, abstractmethod
from typing import Any

class Connection(ABC):
    """Abstract connection interface."""
    
    @abstractmethod
    def connect(self) -> None:
        pass
    
    @abstractmethod
    def execute(self, query: str) -> Any:
        pass

class PostgresConnection(Connection):
    def __init__(self, host: str, database: str):
        self.host = host
        self.database = database
    
    def connect(self) -> None:
        print(f"Connecting to PostgreSQL: {self.host}/{self.database}")
    
    def execute(self, query: str) -> Any:
        return f"PostgreSQL executing: {query}"

class SnowflakeConnection(Connection):
    def __init__(self, account: str, warehouse: str):
        self.account = account
        self.warehouse = warehouse
    
    def connect(self) -> None:
        print(f"Connecting to Snowflake: {self.account}/{self.warehouse}")
    
    def execute(self, query: str) -> Any:
        return f"Snowflake executing: {query}"

class ConnectionFactory:
    """Factory for creating connections."""
    
    @staticmethod
    def create_connection(conn_type: str, **config) -> Connection:
        """Create connection based on type."""
        if conn_type == "postgres":
            return PostgresConnection(
                host=config.get("host", "localhost"),
                database=config["database"]
            )
        elif conn_type == "snowflake":
            return SnowflakeConnection(
                account=config["account"],
                warehouse=config["warehouse"]
            )
        else:
            raise ValueError(f"Unknown connection type: {conn_type}")

# Use factory
conn = ConnectionFactory.create_connection(
    "postgres",
    host="db.example.com",
    database="analytics"
)
conn.connect()
print(conn.execute("SELECT * FROM users"))

### 🔍 Odibi Analysis: Factory Pattern

**Where used:**
- `create_context()` - Creates execution context based on config
- Connection factory - Creates database connections
- Engine factory - Creates compute engines (Spark, Pandas, Polars)

**Benefits:**
- Centralize object creation logic
- Hide complex initialization
- Return interface, not concrete type

## Pattern 3: Strategy Pattern

### ⚡ Example: Execution Engines

In [ ]:
from abc import ABC, abstractmethod

class ExecutionEngine(ABC):
    """Strategy for executing data operations."""
    
    @abstractmethod
    def filter(self, data: Any, condition: str) -> Any:
        pass
    
    @abstractmethod
    def aggregate(self, data: Any, columns: list[str]) -> Any:
        pass

class PandasEngine(ExecutionEngine):
    def filter(self, data: pd.DataFrame, condition: str) -> pd.DataFrame:
        return data.query(condition)
    
    def aggregate(self, data: pd.DataFrame, columns: list[str]) -> pd.DataFrame:
        return data.groupby(columns).size().reset_index(name='count')

class PolarsEngine(ExecutionEngine):
    def filter(self, data: Any, condition: str) -> Any:
        # Simplified - real implementation would parse condition
        return f"Polars filter: {condition}"
    
    def aggregate(self, data: Any, columns: list[str]) -> Any:
        return f"Polars groupby: {columns}"

class DataPipeline:
    """Context that uses an execution engine."""
    
    def __init__(self, engine: ExecutionEngine):
        self.engine = engine
    
    def process(self, data: Any) -> Any:
        """Process using the configured engine."""
        filtered = self.engine.filter(data, "age > 25")
        aggregated = self.engine.aggregate(filtered, ["city"])
        return aggregated

# Use with different strategies
df = pd.DataFrame({
    "name": ["Alice", "Bob", "Charlie"],
    "age": [30, 20, 35],
    "city": ["NYC", "LA", "NYC"]
})

# Same pipeline, different engine
pandas_pipeline = DataPipeline(PandasEngine())
result = pandas_pipeline.process(df)
print("Pandas result:")
print(result)

polars_pipeline = DataPipeline(PolarsEngine())
print(f"\nPolars result: {polars_pipeline.process(df)}")

### 🔍 Odibi Analysis: Strategy Pattern

**Where used:**
- Different execution engines (Spark, Pandas, Polars) with same interface
- Different serialization strategies (JSON, Parquet, CSV)
- Different validation strategies

**Benefits:**
- Swap implementations without changing client code
- Test with mock strategies
- Configure behavior at runtime

## Pattern 4: Builder Pattern

### ⚡ Example: Config Builder

In [ ]:
from dataclasses import dataclass, field
from typing import Optional

@dataclass
class PipelineConfig:
    """Pipeline configuration."""
    name: str
    source: Optional[str] = None
    target: Optional[str] = None
    transforms: list[str] = field(default_factory=list)
    validation_enabled: bool = True
    retry_count: int = 3

class PipelineConfigBuilder:
    """Builder for pipeline configuration."""
    
    def __init__(self, name: str):
        self._config = PipelineConfig(name=name)
    
    def with_source(self, source: str) -> 'PipelineConfigBuilder':
        self._config.source = source
        return self
    
    def with_target(self, target: str) -> 'PipelineConfigBuilder':
        self._config.target = target
        return self
    
    def add_transform(self, transform: str) -> 'PipelineConfigBuilder':
        self._config.transforms.append(transform)
        return self
    
    def with_validation(self, enabled: bool) -> 'PipelineConfigBuilder':
        self._config.validation_enabled = enabled
        return self
    
    def with_retries(self, count: int) -> 'PipelineConfigBuilder':
        self._config.retry_count = count
        return self
    
    def build(self) -> PipelineConfig:
        """Build the final config."""
        if not self._config.source:
            raise ValueError("Source is required")
        if not self._config.target:
            raise ValueError("Target is required")
        return self._config

# Build configuration fluently
config = (
    PipelineConfigBuilder("user_pipeline")
    .with_source("postgres://users")
    .with_target("s3://analytics/users")
    .add_transform("clean_names")
    .add_transform("filter_active")
    .with_validation(True)
    .with_retries(5)
    .build()
)

print(config)

### 🔍 Odibi Analysis: Builder Pattern

**Where used:**
- Config building with fluent interface
- Query builders for different engines
- Pipeline construction

**Benefits:**
- Readable, fluent API
- Validate during construction
- Handle complex object creation

## Pattern 5: Singleton Pattern

### ⚡ Example: Config Manager

In [ ]:
class ConfigManager:
    """Singleton config manager."""
    
    _instance: Optional['ConfigManager'] = None
    _config: dict = {}
    
    def __new__(cls):
        if cls._instance is None:
            cls._instance = super().__new__(cls)
        return cls._instance
    
    def set(self, key: str, value: Any) -> None:
        self._config[key] = value
    
    def get(self, key: str, default: Any = None) -> Any:
        return self._config.get(key, default)
    
    def clear(self) -> None:
        """Clear config (useful for testing)."""
        self._config.clear()

# Same instance everywhere
config1 = ConfigManager()
config1.set("database", "analytics")

config2 = ConfigManager()
print(f"Same instance: {config1 is config2}")  # True
print(f"Database: {config2.get('database')}")  # analytics

### ⚠️ Singleton Anti-Pattern

**Problems with Singleton:**
- Makes testing harder (global state)
- Hidden dependencies
- Tight coupling

**Better Alternative: Dependency Injection**

## Pattern 6: Dependency Injection

### ⚡ Example: Explicit Dependencies

In [ ]:
class Logger:
    def log(self, message: str) -> None:
        print(f"LOG: {message}")

class MetricsCollector:
    def record(self, metric: str, value: float) -> None:
        print(f"METRIC: {metric} = {value}")

# BAD: Hidden dependencies (Singleton)
class PipelineBad:
    def run(self):
        logger = ConfigManager().get("logger")  # Hidden!
        logger.log("Starting")

# GOOD: Explicit dependencies (Dependency Injection)
class Pipeline:
    def __init__(self, logger: Logger, metrics: MetricsCollector):
        self.logger = logger
        self.metrics = metrics
    
    def run(self, data: pd.DataFrame) -> pd.DataFrame:
        self.logger.log("Starting pipeline")
        self.metrics.record("rows", len(data))
        
        # Process data
        result = data.copy()
        
        self.logger.log("Pipeline complete")
        return result

# Easy to test - inject mocks
logger = Logger()
metrics = MetricsCollector()
pipeline = Pipeline(logger=logger, metrics=metrics)

df = pd.DataFrame({"x": [1, 2, 3]})
result = pipeline.run(df)

### 🔍 Odibi Analysis: Dependency Injection

**Where used:**
- Execution context passed explicitly
- Config injected into components
- Connections passed to transforms

**Benefits:**
- Clear dependencies
- Easy to test
- Flexible composition

## 🏗️ Build: Mini Plugin System

Combine patterns to build an extensible plugin system.

In [ ]:
from abc import ABC, abstractmethod
from typing import Dict, Callable, Any

# Strategy Pattern: Plugin interface
class Plugin(ABC):
    @abstractmethod
    def execute(self, data: pd.DataFrame, **kwargs) -> pd.DataFrame:
        pass

# Registry Pattern: Plugin registry
class PluginRegistry:
    _plugins: Dict[str, type[Plugin]] = {}
    
    @classmethod
    def register(cls, name: str):
        def decorator(plugin_class: type[Plugin]):
            cls._plugins[name] = plugin_class
            return plugin_class
        return decorator
    
    @classmethod
    def get(cls, name: str) -> type[Plugin]:
        return cls._plugins[name]

# Factory Pattern: Plugin factory
class PluginFactory:
    @staticmethod
    def create(name: str, **config) -> Plugin:
        plugin_class = PluginRegistry.get(name)
        return plugin_class(**config)

# Builder Pattern: Pipeline builder
class PipelineBuilder:
    def __init__(self):
        self._steps: list[tuple[str, dict]] = []
    
    def add_step(self, plugin_name: str, **config) -> 'PipelineBuilder':
        self._steps.append((plugin_name, config))
        return self
    
    def build(self) -> 'PluginPipeline':
        plugins = [
            PluginFactory.create(name, **config)
            for name, config in self._steps
        ]
        return PluginPipeline(plugins)

# Dependency Injection: Pipeline with explicit dependencies
class PluginPipeline:
    def __init__(self, plugins: list[Plugin]):
        self.plugins = plugins
    
    def execute(self, data: pd.DataFrame) -> pd.DataFrame:
        result = data
        for plugin in self.plugins:
            result = plugin.execute(result)
        return result

# Define plugins
@PluginRegistry.register("uppercase")
class UppercasePlugin(Plugin):
    def __init__(self, column: str):
        self.column = column
    
    def execute(self, data: pd.DataFrame, **kwargs) -> pd.DataFrame:
        result = data.copy()
        result[self.column] = result[self.column].str.upper()
        return result

@PluginRegistry.register("filter")
class FilterPlugin(Plugin):
    def __init__(self, condition: str):
        self.condition = condition
    
    def execute(self, data: pd.DataFrame, **kwargs) -> pd.DataFrame:
        return data.query(self.condition)

# Use the plugin system
pipeline = (
    PipelineBuilder()
    .add_step("uppercase", column="name")
    .add_step("filter", condition="age > 25")
    .build()
)

df = pd.DataFrame({
    "name": ["alice", "bob", "charlie"],
    "age": [30, 20, 35]
})

result = pipeline.execute(df)
print(result)

## ✅ Test: Pattern Testing

In [ ]:
# Test plugin system
def test_plugin_pipeline():
    # Arrange
    df = pd.DataFrame({
        "name": ["alice", "bob"],
        "age": [30, 20]
    })
    
    pipeline = (
        PipelineBuilder()
        .add_step("uppercase", column="name")
        .build()
    )
    
    # Act
    result = pipeline.execute(df)
    
    # Assert
    assert result["name"].tolist() == ["ALICE", "BOB"]
    print("✓ Plugin pipeline test passed")

test_plugin_pipeline()

## 🎯 Summary

| Pattern | Purpose | Odibi Usage |
|---------|---------|-------------|
| Registry | Discover & register components | Transform/function registry |
| Factory | Create objects by type | `create_context()`, connections |
| Strategy | Swap implementations | Different engines (Spark/Pandas) |
| Builder | Fluent construction | Config building, query building |
| Singleton | Single instance (avoid!) | - |
| Dependency Injection | Explicit dependencies | Context, config, connections |

**Key Takeaway**: Use patterns to solve real problems, not for their own sake.